In [176]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

# import package

# model
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torch import optim
from torch.optim.lr_scheduler import StepLR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau

# dataset and transformation
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import models
import torch.nn as nn
import torch.nn.functional as F
import torchvision

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, LSTM, Dropout, ConvLSTM2D, Flatten, TimeDistributed,Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import Callback
from keras.regularizers import l2
from tensorflow.keras.optimizers import Nadam

# display images
from torchvision import utils
import matplotlib.pyplot as plt
%matplotlib inline

# utils
import numpy as np
from torchsummary import summary
import time
import copy
import pandas as pd


from warnings import filterwarnings
filterwarnings('ignore')

import torch
from torch import nn, optim
from torch.backends import cudnn
cudnn.benchmark = True
from randaugment import RandAugment, ImageNetPolicy

import seaborn as sns
plt.rcParams['font.family'] = 'Malgun Gothic'

import tensorflow as tf

In [116]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

cuda:0


In [117]:
tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None )

True

In [118]:
train =pd.read_csv("../data/train_preprocessing2.csv")
submission = pd.read_csv("../data/sample_submission.csv")

In [119]:
train.drop('일교차',inplace=True, axis = 1)
train.loc[train['강수량'] < 0 , '강수량'] = 0

import math
train['강수량'] = train['강수량'].apply(lambda x :  math.log(x+1))

In [120]:
train

,일시,최고기온,최저기온,강수량,평균습도,평균풍속,일조합,일사합,일조율,평균기온,년,월_sin,월_cos,일_sin,일_cos
0,1960-01-01,2.2,-5.2,2.026732,68.3,1.7,6.7,12.562132,55.338502,-1.6,1960,5.000000e-01,0.866025,2.012985e-01,0.979530
1,1960-01-02,1.2,-5.6,0.336472,87.7,1.3,0.0,4.541569,2.040280,-1.9,1960,5.000000e-01,0.866025,3.943559e-01,0.918958
2,1960-01-03,8.7,-2.1,0.000000,81.3,3.0,0.0,5.738938,2.040280,4.0,1960,5.000000e-01,0.866025,5.712682e-01,0.820763
3,1960-01-04,10.8,1.2,0.000000,79.7,4.4,2.6,8.422487,22.723172,7.5,1960,5.000000e-01,0.866025,7.247928e-01,0.688967
4,1960-01-05,1.3,-8.2,1.884740,44.0,5.1,8.2,14.946189,67.270939,-4.6,1960,5.000000e-01,0.866025,8.486443e-01,0.528964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23006,2022-12-27,3.3,-7.3,2.030017,69.8,1.8,8.8,10.250000,91.700000,-2.6,2022,-2.449294e-16,1.000000,-7.247928e-01,0.688967
23007,2022-12-28,0.1,-6.0,0.095310,58.1,2.5,8.7,10.860000,90.600000,-3.3,2022,-2.449294e-16,1.000000,-5.712682e-01,0.820763
23008,2022-12-29,2.1,-7.8,0.000000,56.3,1.7,9.0,10.880000,93.800000,-2.9,2022,-2.449294e-16,1.000000,-3.943559e-01,0.918958
23009,2022-12-30,2.3,-4.4,0.000000,65.6,1.9,7.9,10.840000,82.300000,-1.8,2022,-2.449294e-16,1.000000,-2.012985e-01,0.979530


In [121]:
print(train['평균기온'].max(),train['평균기온'].min())

33.7 -16.4


In [122]:
# Min-Max Scaler 초기화
scaler = MinMaxScaler()

# Min-Max Scaling을 적용할 컬럼들 선택
columns_to_scale = ['최고기온', '최저기온', '강수량', '평균습도', '평균풍속', '일조합', '일사합', '일조율', '평균기온', '년']

# Scaling 적용
train[columns_to_scale] = scaler.fit_transform(train[columns_to_scale])

In [123]:
train.drop('일시',axis = 1 , inplace = True)

In [124]:
train

,최고기온,최저기온,강수량,평균습도,평균풍속,일조합,일사합,일조율,평균기온,년,월_sin,월_cos,일_sin,일_cos
0,0.296992,0.297030,0.348803,0.615385,0.207792,0.489051,0.375213,0.554015,0.295409,0.0,5.000000e-01,0.866025,2.012985e-01,0.979530
1,0.278195,0.289109,0.057907,0.852259,0.155844,0.000000,0.135650,0.020426,0.289421,0.0,5.000000e-01,0.866025,3.943559e-01,0.918958
2,0.419173,0.358416,0.000000,0.774115,0.376623,0.000000,0.171414,0.020426,0.407186,0.0,5.000000e-01,0.866025,5.712682e-01,0.820763
3,0.458647,0.423762,0.000000,0.754579,0.558442,0.189781,0.251568,0.227490,0.477046,0.0,5.000000e-01,0.866025,7.247928e-01,0.688967
4,0.280075,0.237624,0.324366,0.318681,0.649351,0.598540,0.446421,0.673475,0.235529,0.0,5.000000e-01,0.866025,8.486443e-01,0.528964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23006,0.317669,0.255446,0.349368,0.633700,0.220779,0.642336,0.306153,0.918044,0.275449,1.0,-2.449294e-16,1.000000,-7.247928e-01,0.688967
23007,0.257519,0.281188,0.016403,0.490842,0.311688,0.635036,0.324373,0.907032,0.261477,1.0,-2.449294e-16,1.000000,-5.712682e-01,0.820763
23008,0.295113,0.245545,0.000000,0.468864,0.207792,0.656934,0.324970,0.939068,0.269461,1.0,-2.449294e-16,1.000000,-3.943559e-01,0.918958
23009,0.298872,0.312871,0.000000,0.582418,0.233766,0.576642,0.323775,0.823937,0.291417,1.0,-2.449294e-16,1.000000,-2.012985e-01,0.979530


In [125]:
window_size = 358*2
prediction_length = 358
X_train = []
y_train = []

# 윈도우에 따른 데이터 분할
for i in range(window_size, len(train)-357):
    X_train.append(train.iloc[i-window_size:i])
    y_train.append(train['평균기온'].iloc[i:i+358].values)  # 358일치 평균기온을 배열로 추가

In [126]:
X_train, y_train = np.array(X_train), np.array(y_train)

In [127]:
print(X_train.shape,y_train.shape)

(21938, 716, 14) (21938, 358)


In [160]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=3, min_lr=0.00001,verbose=1)

class LearningRateTracker(Callback):
    def on_epoch_begin(self, epoch, logs=None):
        lr = self.model.optimizer.lr
        # 학습률을 실제 값으로 변환하고 float 타입으로 캐스팅
        current_lr = float(K.eval(lr))
        # 반올림
        rounded_lr = round(current_lr, 6)
        print(f'에포크 {epoch + 1}: 현재 학습률 = {rounded_lr}')

# 콜백 인스턴스 생성
lr_tracker = LearningRateTracker()

class CustomCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        val_loss = logs.get('val_loss')
        print(f"에포크 {epoch + 1}: 검증 손실 = {val_loss:.5f}")

custom_callback = CustomCallback()


In [193]:
# 모델 구축
model = Sequential()
model.add(Bidirectional(GRU(units=32, return_sequences=True), input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Bidirectional(GRU(units=32)))
model.add(Dense(units=358))

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.0002), loss='mean_squared_error')

# 모델 요약 출력
model.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_4 (Bidirection (None, 716, 64)           9216      
_________________________________________________________________
bidirectional_5 (Bidirection (None, 64)                18816     
_________________________________________________________________
dense_18 (Dense)             (None, 358)               23270     
Total params: 51,302
Trainable params: 51,302
Non-trainable params: 0
_________________________________________________________________


In [194]:
# 모델 훈련
history = model.fit(X_train, y_train,epochs=10, batch_size=32, verbose=2,workers=6)

Epoch 1/10
686/686 - 94s - loss: 0.0438
Epoch 2/10
686/686 - 92s - loss: 0.0048
Epoch 3/10
686/686 - 93s - loss: 0.0046
Epoch 4/10
686/686 - 98s - loss: 0.0045
Epoch 5/10
686/686 - 103s - loss: 0.0044
Epoch 6/10
686/686 - 104s - loss: 0.0043
Epoch 7/10
686/686 - 105s - loss: 0.0042
Epoch 8/10
686/686 - 105s - loss: 0.0042
Epoch 9/10
686/686 - 104s - loss: 0.0042
Epoch 10/10
686/686 - 103s - loss: 0.0042


In [195]:
predict_data = X_train[-1].reshape(1, -1, 14)

In [196]:
pred = model.predict(predict_data)

In [197]:
# MinMaxScaler의 역변환을 수행하는 함수 정의
def inverse_minmax_scaling(scaled_data, min_val, max_val):
    return (scaled_data * (max_val - min_val)) + min_val

# 최소값과 최대값 정의
min_val = -16.4
max_val = 33.7

# 역변환 수행
pred = inverse_minmax_scaling(pred, min_val, max_val)

In [198]:
pred = pred[-1].reshape(-1,)

In [199]:
pred

array([-3.23397827e+00, -1.86255550e+00, -1.52338791e+00, -1.71996498e+00,
       -1.93726826e+00, -2.38850307e+00, -2.10128403e+00, -1.64516544e+00,
       -1.65165138e+00, -1.47793102e+00, -1.64198780e+00, -1.23384571e+00,
       -1.50688839e+00, -1.33997822e+00, -1.45185852e+00, -1.49444771e+00,
       -1.06869888e+00, -1.63847446e+00, -1.19088745e+00, -9.25667763e-01,
       -7.73042679e-01, -1.27964020e+00, -1.19408989e+00, -1.28328228e+00,
       -9.85093117e-01, -1.22547817e+00, -6.50426865e-01, -5.13263702e-01,
       -1.75064754e+00, -8.34344864e-01, -1.04347229e-01, -3.95103455e-01,
       -5.82758904e-01, -9.11547661e-01,  4.36113358e-01,  1.24931335e-03,
        7.81448364e-01,  5.20452499e-01,  9.78450775e-01,  9.52989578e-01,
        1.32740974e+00,  1.12363243e+00,  1.04484177e+00,  1.60214996e+00,
        1.25073242e+00,  1.66740417e+00,  2.73576164e+00,  2.94528008e+00,
        2.99255371e+00,  3.50020409e+00,  2.80494499e+00,  2.65245819e+00,
        3.50481796e+00,  

In [200]:
sub = pd.read_csv('../data/sample_submission.csv')
sub['평균기온'] = pred

sub.to_csv('../submission/BiGRU_3.csv', index=False)